In [ ]:
# import required packages
from __future__ import print_function, division
#import cv2
import dlib
import time
from skimage import io
import os
import shutil
import torch
import pandas as pd
import torchvision
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.stats.api as sms
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from os import listdir
from os.path import isfile, join
from collections import namedtuple
from skimage.transform import rescale
from skimage.transform import resize
import sys
import glob
import PIL
import os
import shutil
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
import math
import PIL
import imgaug as ia
from imgaug import augmenters as iaa
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from pathlib import Path
from collections import Counter
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
print(os.getcwd())
#np.random.seed(198467)
torch.cuda.empty_cache()

In [2]:
frame = pd.read_csv('ExpW_OpenFace_result_with_expression_gender_race_age.csv')
frame = frame[['ImageName', 'Expression', 'gender_preds']]
frame.gender_preds = frame.gender_preds.apply(lambda x: 1 if x == "Male" else 0)
frame = frame[['ImageName', 'Expression', 'gender_preds']]
frame.Expression = frame.Expression.apply(lambda x: 1 if x == 3 else 0)
frame.head()

,ImageName,Expression,gender_preds
0,angry_actor_104_0.jpg,0,1
1,angry_actor_109_0.jpg,0,1
2,angry_actor_120_1.jpg,0,1
3,angry_actor_13_0.jpg,1,0
4,angry_actor_132_0.jpg,0,0


In [3]:
frame.shape

(88600, 3)

In [4]:
frame.Expression.value_counts()

0    59707
1    28893
Name: Expression, dtype: int64

In [5]:
frame.gender_preds.value_counts()

1    61108
0    27492
Name: gender_preds, dtype: int64

In [6]:
frame['ImageName'] = frame['ImageName'].apply(lambda x: '/data/ExpW_cropped_by_their_coordinates/' + x) # change ImageName to full path
frame_copy = frame.copy()

In [7]:
result_rows_list = []

In [8]:
for k in range(1,6):

    # Split into train/validation/test sets
    frame = frame_copy.sample(n = 20000, random_state = k).reset_index(drop=True) # shuffle data frame
    n_images = len(frame)
    n_train = int(0.8 * n_images)
    n_val = int((n_images - n_train) / 2)
    n_test = n_images - n_train - n_val

    train_frame = frame[0 : n_train].reset_index(drop=True)
    val_frame = frame[n_train : n_train + n_val].reset_index(drop=True)
    test_frame = frame[n_train + n_val : ].reset_index(drop=True)

    print("{} train faces, {} validation faces, {} test faces".format(len(train_frame), len(val_frame), len(test_frame)))

    # Data loaders and transforms for training

    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
    # image.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image.
    class ImgAugTransform:

        def __init__(self):
            self.aug = iaa.Sequential(
            [
                #
                # Apply the following augmenters to most images.
                #
                iaa.Fliplr(0.5), # horizontally flip 50% of all images
                #iaa.Flipud(0.2), # vertically flip 20% of all images

                # crop some of the images by 0-10% of their height/width
                sometimes(iaa.Crop(percent=(0, 0.05))),

                # Apply affine transformations to some of the images
                # - scale to 80-120% of image height/width (each axis independently)
                # - translate by -20 to +20 relative to height/width (per axis)
                # - rotate by -45 to +45 degrees
                # - shear by -16 to +16 degrees
                # - order: use nearest neighbour or bilinear interpolation (fast)
                # - mode: use any available mode to fill newly created pixels
                #         see API or scikit-image for which modes are available
                # - cval: if the mode is constant, then use a random brightness
                #         for the newly created pixels (e.g. sometimes black,
                #         sometimes white)
                iaa.Affine(
                    scale={"x": (1, 1.1), "y": (1, 1.1)}, # scale images to 80-120% of their size, individually per axis
                    translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -10 to +10 percent (per axis)
                    rotate=(-15, 15), # rotate by -15 to +15 degrees
                    shear=(-8, 8), # shear by -8 to +8 degrees
                    order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                    #cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                    mode=['edge'] # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                ),

                #
                # Execute 0 to 5 of the following (less important) augmenters per
                # image. Don't execute all of them, as that would often be way too
                # strong.
                #
                iaa.SomeOf((0, 5),
                    [
                        # Convert some images into their superpixel representation,
                        # sample between 20 and 200 superpixels per image, but do
                        # not replace all superpixels with their average, only
                        # some of them (p_replace).
                        sometimes(
                            iaa.Superpixels(
                                p_replace=(0, 0.1),
                                n_segments=(50, 200)
                            )
                        ),

                        # Blur each image with varying strength using
                        # gaussian blur (sigma between 0 and 3.0),
                        # average/uniform blur (kernel size between 2x2 and 7x7)
                        # median blur (kernel size between 3x3 and 11x11).
                        iaa.OneOf([
                            iaa.GaussianBlur((0, 3.0)),
                            iaa.AverageBlur(k=(2, 7)),
                            iaa.MedianBlur(k=(3, 11)),
                        ]),

                        # Sharpen each image, overlay the result with the original
                        # image using an alpha between 0 (no sharpening) and 1
                        # (full sharpening effect).
                        iaa.Sharpen(alpha=(0, 0.3), lightness=(0.75, 1.5)),

                        # Same as sharpen, but for an embossing effect.
                        iaa.Emboss(alpha=(0, 0.3), strength=(0, 2)),

                        # Search in some images either for all edges or for
                        # directed edges. These edges are then marked in a black
                        # and white image and overlayed with the original image
                        # using an alpha of 0 to 0.7.
                        sometimes(iaa.OneOf([
                            iaa.EdgeDetect(alpha=(0, 0.3)),
                            iaa.DirectedEdgeDetect(
                                alpha=(0, 0.3), direction=(0.0, 1.0)
                            ),
                        ])),

                        # Add gaussian noise to some images.
                        # In 50% of these cases, the noise is randomly sampled per
                        # channel and pixel.
                        # In the other 50% of all cases it is sampled once per
                        # pixel (i.e. brightness change).
                        iaa.AdditiveGaussianNoise(
                            loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                        ),

                        # Either drop randomly 1 to 10% of all pixels (i.e. set
                        # them to black) or drop them on an image with 2-5% percent
                        # of the original size, leading to large dropped
                        # rectangles.
                        iaa.OneOf([
                            iaa.Dropout((0.01, 0.02), per_channel=0.5),
                            #iaa.CoarseDropout(
                            #    (0.03, 0.15), size_percent=(0.02, 0.05),
                            #    per_channel=0.2
                            #),
                        ]),

                        # Invert each image's chanell with 5% probability.
                        # This sets each pixel value v to 255-v.
                        #iaa.Invert(0.05, per_channel=True), # invert color channels

                        # Add a value of -10 to 10 to each pixel.
                        iaa.Add((-15, 15), per_channel=0.5),

                        # Change brightness of images (50-150% of original value).
                        iaa.Multiply((0.75, 1.25), per_channel=0.5),

                        # Improve or worsen the contrast of images.
                        iaa.ContrastNormalization((0.75, 1.75), per_channel=0.5),

                        # Convert each image to grayscale and then overlay the
                        # result with the original with random alpha. I.e. remove
                        # colors with varying strengths.
                        iaa.Grayscale(alpha=(0.0, 1.0)),

                        # In some images move pixels locally around (with random
                        # strengths).
                        #sometimes(
                        #    iaa.ElasticTransformation(alpha=(0.1, 0.2), sigma=0.25)
                        #),

                        # In some images distort local areas with varying strength.
                        sometimes(iaa.PiecewiseAffine(scale=(0.005, 0.01)))
                    ],
                    # do all of the above augmentations in random order
                    random_order=True
                )
            ],
            # do all of the above augmentations in random order
            random_order=True
        )

        def __call__(self, img):

            img = np.array(img)
            return self.aug.augment_image(img)

    class ImageDataset(Dataset):

        def __init__(self, data_frame, transform=None):

            self.data_frame = data_frame
            self.transform = transform

        def __len__(self):

            return len(self.data_frame)

        def __getitem__(self, idx):
            #idx is index from dataset
            #This is a mapping from your data_frame to the output of the mode
            img_name = self.data_frame.loc[idx, 'ImageName']

            expression = self.data_frame.loc[idx, 'Expression']
            target = self.data_frame.loc[idx, ['Expression', 'gender_preds']]

            # read image as ndarray, H*W*C
            image = dlib.load_rgb_image(img_name)       
            image = cv2.resize(image, (224,224)) # resize the image to 224x224 for the ResNet Model

            if self.transform:
                image = self.transform(image)

            # transform label to torch tensor
            # This sets the order of the label
            return (image, torch.from_numpy(np.asarray(target, dtype=np.float32)))


    transform_train_data = transforms.Compose([
        ImgAugTransform(),
        lambda x: PIL.Image.fromarray(x),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    transformed_train_dataset = ImageDataset(data_frame=train_frame,
                                               transform=transform_train_data
                                               )

    train_dataloader = DataLoader(transformed_train_dataset, batch_size=64,
                            shuffle=False, num_workers=8)


    transform_val_data = transforms.Compose(([transforms.ToPILImage(), 
                                          transforms.ToTensor(), 
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                              ]))

    transformed_val_dataset = ImageDataset(data_frame=val_frame,
                                               transform=transform_val_data
                                               )

    val_dataloader = DataLoader(transformed_val_dataset, batch_size=64,
                            shuffle=False, num_workers=8)

    transform_test_data = transforms.Compose(([transforms.ToPILImage(), 
                                          transforms.ToTensor(), 
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                              ]))

    transformed_test_dataset = ImageDataset(data_frame=test_frame,
                                               transform=transform_test_data
                                               )

    test_dataloader = DataLoader(transformed_test_dataset, batch_size=64,
                            shuffle=False, num_workers=8)

    # Training

    torch.cuda.is_available()

    dataloaders = {'train': train_dataloader, 'test': test_dataloader}
    dataset_sizes = {'train': len(transformed_train_dataset), 'test': len(transformed_test_dataset)}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    

    # utils.py
    import json
    import pickle
    from sklearn.metrics import average_precision_score

    def save_pkl(pkl_data, save_path):
        with open(save_path, 'wb') as f:
            pickle.dump(pkl_data, f)

    def load_pkl(load_path):
        with open(load_path, 'rb') as f:
            pkl_data = pickle.load(f)
        return pkl_data

    def save_json(json_data, save_path):
        with open(save_path, 'w') as f:
            json.dump(json_data, f)

    def load_json(load_path):
        with open(load_path, 'r') as f:
            json_data = json.load(f)
        return json_data

    def save_state_dict(state_dict, save_path):
        torch.save(state_dict, save_path)

    def creat_folder(path):
        if not os.path.exists(path):
            os.makedirs(path)

    def set_random_seed(seed_number):
        torch.manual_seed(seed_number)
        np.random.seed(seed_number)

    def write_info(filename, info):
        with open(filename, 'w') as f:
            f.write(info)

    def compute_weighted_AP(target, predict_prob, class_weight_list):
        per_class_AP = []
        for i in range(target.shape[1] - 1):
            class_weight = target[:, i]*class_weight_list[i] \
                           + (1-target[:, i])*np.ones(class_weight_list[i].shape)
            per_class_AP.append(average_precision_score(target[:, i], predict_prob[:], 
                                    sample_weight=class_weight))

        return per_class_AP

    def compute_mAP(per_class_AP):
        return np.mean([per_class_AP[idx] for idx in [0]])

    def compute_class_weight(target):
        domain_label = target[:, -1]
        per_class_weight = []

        for i in range(target.shape[1]-1):
            class_label = target[:, i]
            cp = class_label.sum() # class is positive
            cn = target.shape[0] - cp # class is negative
            cn_dn = ((class_label + domain_label)==0).sum() # class is negative, domain is negative
            cn_dp = ((class_label - domain_label)==-1).sum()
            cp_dn = ((class_label - domain_label)==1).sum()
            cp_dp = ((class_label + domain_label)==2).sum()

            per_class_weight.append(
                (class_label*cp + (1-class_label)*cn) / 
                    (2*(
                        (1-class_label)*(1-domain_label)*cn_dn
                        + (1-class_label)*domain_label*cn_dp
                        + class_label*(1-domain_label)*cp_dn
                        + class_label*domain_label*cp_dp
                       )
                    )
            )
        return per_class_weight

    # celeba_domain_discriminative.py
    class ResNet50(nn.Module):    
        def __init__(self, n_classes, pretrained, hidden_size=2048, dropout=0.5):
            super().__init__()
            self.resnet = torchvision.models.resnet50(pretrained=pretrained)                
            self.resnet.fc = nn.Linear(2048, hidden_size)
            self.fc = nn.Linear(hidden_size, n_classes)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(dropout)        

        def require_all_grads(self):
            for param in self.parameters():
                param.requires_grad = True

        def forward(self, x):
            features = self.resnet(x)
            outputs = self.fc(self.dropout(self.relu(features)))

            return outputs, features
    class CelebaDomainDiscriminative():
        def __init__(self):
            self.epoch = 0

            self.train_loader = train_dataloader    
            self.dev_loader = val_dataloader        
            self.test_loader = test_dataloader


            self.dev_target = val_frame.iloc[:,[1,2]].to_numpy()
            self.dev_class_weight = compute_class_weight(self.dev_target)
            self.test_target = test_frame.iloc[:,[1,2]].to_numpy()
            self.test_class_weight = compute_class_weight(self.test_target)


            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

            self.network = ResNet50(n_classes=2, pretrained=True).to(self.device)

            self.optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.network.parameters()), 
                                      lr=1e-4)

            self.best_dev_mAP = 0.



        def forward(self, x):
            out, feature = self.network(x)
            return out, feature

        def state_dict(self):
            state_dict = {
                'model': self.network.state_dict(),
                'optimizer': self.optimizer.state_dict(),
                'epoch': self.epoch
            }
            return state_dict

        def _criterion(self, output, target):
            domain_label = target[:, -1:]
            two_n_target = torch.cat([target[:, :-1]*domain_label, 
                                      target[:, :-1]*(1-domain_label)],
                                     dim=1)
            return F.binary_cross_entropy_with_logits(output, two_n_target)

        def inference(self, output):
            """Inference method: sum the probability from two domains"""

            predict_prob = torch.sigmoid(output)
            class_num = predict_prob.shape[1] // 2
            return predict_prob[:, :class_num] + predict_prob[:, class_num:]   



        def _train(self, loader):
            """Train the model for one epoch"""

            self.network.train()

            train_loss = 0
            total = 0
            class_correct = 0
            for i, (images, targets) in enumerate(loader):
                images, targets = images.to(self.device), targets.to(self.device)
                expression = targets[:,0]
                expression = expression.to(device, dtype=torch.long)  

                self.optimizer.zero_grad()

                outputs, _ = self.forward(images)
                outputs = torch.squeeze(outputs)

                loss = self._criterion(outputs, targets)
                loss.backward()
                self.optimizer.step()

                total += expression.size(0)

                predict_prob = torch.squeeze(self.inference(outputs))
                class_predicted = torch.where(predict_prob >= 0.5, torch.ones_like(predict_prob), torch.zeros_like(predict_prob)).long()
                class_correct += torch.sum(class_predicted.cpu() == expression.cpu())  

                train_loss += loss.item()
                #self.log_result('Train iteration', {'loss': loss.item()},
                #                len(loader)*self.epoch + i)

                #if self.print_freq and (i % self.print_freq == 0):
            print('Training epoch {}: [{}|{}], loss:{}, accuracy:{}'.format(
                          self.epoch, i+1, len(loader), loss.item(), 100.*class_correct/total))

            #self.log_result('Train epoch', {'loss': train_loss/len(loader)}, self.epoch)
            self.epoch += 1



        def _test(self, loader):
            """Compute model output on test set"""

            self.network.eval()

            test_loss = 0
            total = 0
            class_correct = 0
            output_list = []
            feature_list = []
            with torch.no_grad():
                for i, (images, targets) in enumerate(loader):
                    images, targets = images.to(self.device), targets.to(self.device)
                    expression = targets[:,0]
                    expression = expression.to(device, dtype=torch.long)      

                    outputs, features = self.forward(images)
                    outputs = torch.squeeze(outputs)
                    loss = self._criterion(outputs, targets)
                    test_loss += loss.item()

                    total += expression.size(0)

                    predict_prob = torch.squeeze(self.inference(outputs))
                    class_predicted = torch.where(predict_prob >= 0.5, torch.ones_like(predict_prob), torch.zeros_like(predict_prob)).long()
                    class_correct += torch.sum(class_predicted.cpu() == expression.cpu()) 

                    output_list.append(outputs)
                    feature_list.append(features)

            return test_loss, torch.cat(output_list), 100.*class_correct/total, torch.cat(feature_list)



        def public_test(self, images):
            """Compute model output on test set"""

            self.network.eval()


            outputs, _ = self.forward(images)
            dev_predict = self.inference(outputs)

            return dev_predict


        def train(self):
            """Train the model for one epoch, evaluate on validation set and 
            save the best model
            """

            start_time = datetime.now()
            learning_rates = [1e-4, 1e-5, 1e-6]
            epochs = [8,3,2]

            for learning_rate, epoch in zip(learning_rates, epochs):
                print()
                print('learning rate:', learning_rate)
                print('epoch number:', epoch)
                self.optimizer = torch.optim.Adam( 
                                      params=filter(lambda p: p.requires_grad, self.network.parameters()), 
                                      lr=learning_rate, weight_decay=1e-5)
                for i in range(epoch):
                    print()
                    self._train(self.train_loader)

                    dev_loss, dev_output, dev_class_accuarcy, _ = self._test(self.dev_loader)
                    dev_predict_prob = self.inference(dev_output).cpu().detach().numpy()
                    dev_per_class_AP = compute_weighted_AP(self.dev_target, dev_predict_prob, 
                                                                 self.dev_class_weight)
                    dev_mAP = compute_mAP(dev_per_class_AP)

                    print('Testing epoch, class loss:{},  class accuracy: {}, mAP: {}'
                              .format(dev_loss/len(self.dev_loader), dev_class_accuarcy, dev_mAP))


                    #self.log_result('Dev epoch', {'loss': dev_loss/len(self.dev_loader), 'mAP': dev_mAP},
                    #                self.epoch)
                    if dev_mAP > self.best_dev_mAP:
                        self.best_dev_mAP = dev_mAP
                        #save_state_dict(self.state_dict(), os.path.join("./test/", 'best_domain_discriminative.pth'))

                save_state_dict(self.state_dict(), os.path.join("./", 'domain_descriminative_' + str(k) + '.pth'))

            duration = datetime.now() - start_time
            print('Finish training epoch {}, dev mAP: {}, dev class accuracy: {}, time used: {}'
                  .format(self.epoch, dev_mAP, dev_class_accuarcy, duration))


        def test(self):
            # Test and save the result
            state_dict = torch.load(os.path.join("./", 'domain_descriminative_' + str(k) + '.pth'))
            self.network.load_state_dict(state_dict['model'])


            test_loss, test_output, test_accuracy, test_feature = self._test(self.test_loader)
            test_predict_prob = self.inference(test_output).cpu().detach().numpy()
            test_per_class_AP = utils.compute_weighted_AP(self.test_target, test_predict_prob, 
                                                         self.test_class_weight)
            test_mAP = utils.compute_mAP(test_per_class_AP)

            print('mAP', test_mAP)



    model = CelebaDomainDiscriminative()
    model.train()



    # Evaluate on Test Set

    # predict on test

    #model = CelebaDomainDiscriminative()
    #model.network.load_state_dict(torch.load('./test/best_domain_discriminative.pth')['model'])

    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    face_names = []
    scores = []
    preds = []

    true_labels = []

    gender_preds = []
    true_gender_labels = []

    for index, row in test_frame.iterrows():

        if index % 200 == 0:
            print(index)

        image_name = row['ImageName']

        image = dlib.load_rgb_image(image_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)
        image = image.to(device)

        outputs = model.public_test(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        score = outputs * 1

        pred = (score>=0.5)*1


        face_names.append(image_name)
        scores.append(score)
        preds.append(pred)

        true_labels.append(row['Expression'])
        true_gender_labels.append(row['gender_preds'])

    test_result = pd.DataFrame(list(zip(face_names, scores, preds, true_labels, true_gender_labels)), 
                 columns = ['ImageName', 'ExpressionScore', 'Prediction', 'Expression', 'Gender'])
    test_result.head(10)

    # Accuracy
    test_result['CorrectOrNot'] = (test_result.Prediction == test_result.Expression)


    dict_row = {}
    dict_row['model_random_state'] = k
    dict_row['test_accuracy'] = test_result.CorrectOrNot.mean()
    dict_row['test_male_true_proportion'] = (test_result[test_result.Gender == 1].Expression==1).mean()
    dict_row['test_female_true_proportion'] = (test_result[test_result.Gender == 0].Expression==1).mean()
    dict_row['test_male_predicted_proportion'] = (test_result[test_result.Gender == 1].Prediction==1).mean()
    dict_row['test_female_predicted_proportion'] = (test_result[test_result.Gender == 0].Prediction==1).mean()
    dict_row['test_male_average_score'] = test_result[test_result.Gender == 1].ExpressionScore.mean()
    dict_row['test_female_average_score'] = test_result[test_result.Gender == 0].ExpressionScore.mean()



    # Apply on Chicago Face

    chicago_df = pd.read_csv('ChicagoFace_selected_evaluation_set2.csv').iloc[:,0:4]


    def expression_string_to_num(s):
        if s == "Happy":
            return 3
        elif s == "Angry":
            return 0
        elif s == "Fear":
            return 2
        else:
            return 6

    chicago_df["Expression_num"] = chicago_df.Expression.apply(expression_string_to_num)


    chicago_df["happy"] = chicago_df.Expression_num.apply(lambda x: 1 if x == 3 else 0)



    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    face_names = []
    scores = []
    preds = []

    true_labels = []

    for index, row in chicago_df.iterrows():

        #if index >= 100: break

        if index % 100 == 0:
            print(index)

        image_name = row['ImageName']


        image = dlib.load_rgb_image(image_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)
        image = image.to(device)

        outputs = model.public_test(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        score = outputs * 1

        pred = (score>=0.5)*1

        face_names.append(image_name)
        scores.append(score)
        preds.append(pred)

        true_labels.append(row['happy'])

    chicago_result = pd.DataFrame([face_names, true_labels, preds, scores]).T

    chicago_result.columns = ['ImageName', 'Expression_num', 'expression_preds', 'expression_scores']

    chicago_result["happiness_score"] = chicago_result.expression_scores.apply(lambda x: x)

    num_happy = (chicago_df.happy == 1).sum()
    
    threshold = chicago_result.sort_values(by = ['happiness_score'], ascending = False).reset_index(drop = True).iloc[num_happy]['happiness_score']

    chicago_result['expression_preds_relabeled'] = chicago_result.happiness_score.apply(lambda x : 1 if x > threshold else 0)

    chicago_result['CorrectOrNot'] = (chicago_result.expression_preds_relabeled == chicago_result.Expression_num)
    dict_row['selected_2_chicago_accuracy'] = chicago_result.CorrectOrNot.mean()

    chicago_df_merged = pd.merge(chicago_df, chicago_result, on = ['ImageName'], how = 'left')
    chicago_df_merged.columns = ['ImageName', 'Gender', 'Race', 'Expression', 'Expression_num_6',
           'happy', 'Expression_num', 'expression_preds', 'expression_scores',
           'happiness_score','expression_preds_relabeled','CorrectOrNot']


    chicago_male_frame = chicago_df_merged.loc[chicago_df_merged['Gender'] == 'M']
    chicago_female_frame = chicago_df_merged.loc[chicago_df_merged['Gender'] == 'F']

    # Accuracy between males and females
    dict_row['selected_2_chicago_accuracy_male'] = (chicago_male_frame.expression_preds_relabeled == chicago_male_frame.Expression_num).mean()
    dict_row['selected_2_chicago_accuracy_female'] = (chicago_female_frame.expression_preds_relabeled == chicago_female_frame.Expression_num).mean()

    # True proportion
    dict_row['selected_2_chicago_true_proportion'] = chicago_male_frame.happy.mean()

    # Prediction proportion
    dict_row['selected_2_chicago_male_predicted_proportion'] = chicago_male_frame.expression_preds_relabeled.mean()
    dict_row['selected_2_chicago_female_predicted_proportion'] = chicago_female_frame.expression_preds_relabeled.mean()
    dict_row['selected_2_chicago_bias'] = chicago_female_frame.expression_preds_relabeled.mean() - chicago_male_frame.expression_preds_relabeled.mean()

    # Prediction proportion
    dict_row['selected_2_chicago_male_predicted_proportion_raw'] = chicago_male_frame.expression_preds.mean()
    dict_row['selected_2_chicago_female_predicted_proportion_raw'] = chicago_female_frame.expression_preds.mean()
    dict_row['selected_2_chicago_bias_raw'] = chicago_female_frame.expression_preds.mean() - chicago_male_frame.expression_preds.mean()

    # Average Happiness Score
    dict_row['selected_2_chicago_male_score'] = chicago_male_frame.happiness_score.mean()
    dict_row['selected_2_chicago_female_score'] = chicago_female_frame.happiness_score.mean()

    # Average Happiness Score among neutral faces
    dict_row['selected_2_chicago_male_score_neutral_faces'] = chicago_male_frame[chicago_male_frame.Expression_num_6 == 6].happiness_score.mean()
    dict_row['selected_2_chicago_female_score_neutral_faces'] = chicago_female_frame[chicago_female_frame.Expression_num_6 == 6].happiness_score.mean()


    result_rows_list.append(dict_row)
    results_df = pd.DataFrame(result_rows_list) 
    print(results_df)
    results_df.to_csv('model_evaluation_result.csv', index = False)

    del model
    torch.cuda.empty_cache()

16000 train faces, 2000 validation faces, 2000 test faces

learning rate: 0.0001
epoch number: 8

Training epoch 0: [250|250], loss:0.32185983657836914, accuracy:79.64375305175781
Testing epoch, class loss:0.2962785419076681,  class accuracy: 84.5999984741211, mAP: 0.8361157534462839

Training epoch 1: [250|250], loss:0.20260904729366302, accuracy:88.61250305175781
Testing epoch, class loss:0.3513608225621283,  class accuracy: 82.80000305175781, mAP: 0.8093853212205271

Training epoch 2: [250|250], loss:0.22469685971736908, accuracy:92.85624694824219
Testing epoch, class loss:0.5256837476044893,  class accuracy: 79.4000015258789, mAP: 0.7635441944354586

Training epoch 3: [250|250], loss:0.15213653445243835, accuracy:95.21875
Testing epoch, class loss:0.5689431373029947,  class accuracy: 81.5999984741211, mAP: 0.8173542113821959

Training epoch 4: [250|250], loss:0.06335818767547607, accuracy:96.89375305175781
Testing epoch, class loss:0.6222376180812716,  class accuracy: 80.5, mAP: 0.


learning rate: 0.0001
epoch number: 8

Training epoch 0: [250|250], loss:0.27760517597198486, accuracy:79.15625
Testing epoch, class loss:0.30819626012817025,  class accuracy: 83.8499984741211, mAP: 0.8256309110345023

Training epoch 1: [250|250], loss:0.19697371125221252, accuracy:88.03125
Testing epoch, class loss:0.3212581258267164,  class accuracy: 84.8499984741211, mAP: 0.8397830690060425

Training epoch 2: [250|250], loss:0.08763322234153748, accuracy:92.9375
Testing epoch, class loss:0.42838418763130903,  class accuracy: 85.30000305175781, mAP: 0.8584976374291029

Training epoch 3: [250|250], loss:0.08200004696846008, accuracy:95.23750305175781
Testing epoch, class loss:0.44797319592908025,  class accuracy: 85.75, mAP: 0.8323542841899182

Training epoch 4: [250|250], loss:0.060540106147527695, accuracy:96.5250015258789
Testing epoch, class loss:0.5247954297810793,  class accuracy: 85.69999694824219, mAP: 0.8486544596920081

Training epoch 5: [250|250], loss:0.01660476066172123,

Testing epoch, class loss:0.7317694490775466,  class accuracy: 86.0999984741211, mAP: 0.8516114302820148
Finish training epoch 13, dev mAP: 0.8516114302820148, dev class accuracy: 86.0999984741211, time used: 0:30:23.964512
0
200
400
600
800
1000
1200
1400
1600
1800
0
100
200
300
400
500
600
700
800
900
1000
1100
0
100
200
300
400
500
0
100
200
300
400
500
600
700
   model_random_state  test_accuracy  test_male_true_proportion  \
0                   1         0.8410                   0.302895   
1                   2         0.8455                   0.319911   
2                   3         0.8565                   0.295570   
3                   4         0.8620                   0.288262   

   test_female_true_proportion  test_male_predicted_proportion  \
0                     0.398162                        0.271715   
1                     0.409712                        0.266965   
2                     0.356340                        0.265795   
3                     0.397554   